In [1]:
import torch
import gc

def free_gpu_memory():
    # Clear unused objects by forcing garbage collection
    gc.collect()

    # Empty the CUDA cache to release unused GPU memory
    torch.cuda.empty_cache()

    # Print current GPU memory usage for monitoring
    print(f"Current allocated GPU memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Current cached GPU memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

free_gpu_memory()

Current allocated GPU memory: 0.00 MB
Current cached GPU memory: 0.00 MB


In [ ]:
!nvidia-smi

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the Qwen model and tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load a Question Answering dataset
sft_dataset = load_dataset("kishore-s-15/curriculum_compass_sft_dataset")["train"]

# Preprocess the dataset
def preprocess_function(examples):
    
    def get_inputs(examples):
        inputs = []
        for question, context in zip(examples["question"], examples["context"]):
            inp = f"""
            Query:
            {question}

            Context:
            {context}
            """

            inputs.append(inp)

        return inputs
            
    inputs = get_inputs(examples)
    targets = examples["response"]
    model_inputs = tokenizer(inputs, max_length=4098*2, truncation=True, padding="max_length")

    # Tokenize the targets
    labels = tokenizer(targets, max_length=4098*2, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# Tokenize the datasets
tokenized_datasets = sft_dataset.map(preprocess_function, batched=True, remove_columns=["question", "context", "response"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./qwen_qa_finetune",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    # save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=0,
    fp16=True,  # Enable mixed precision training if using GPU
    push_to_hub=False
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./qwen_qa_finetune")
tokenizer.save_pretrained("./qwen_qa_finetune")

/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/i

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 31.74 GiB of which 15.56 MiB is free. Process 128540 has 20.43 GiB memory in use. Including non-PyTorch memory, this process has 11.29 GiB memory in use. Of the allocated memory 10.50 GiB is allocated by PyTorch, and 434.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)